In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [3]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content='I am sorry, but I am unable to provide real-time weather information. I recommend checking a reliable weather website or app for the most up-to-date forecast for Munich.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 17, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CKETz8ESYmk1XMZyWaqsyViXcOoFt', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--fa16289e-bb7e-4d9f-a673-ba1136e19e6f-0', usage_metadata={'input_tokens': 17, 'output_tokens': 34, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "munich" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "munich" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."


tools = [get_weather, check_seating_availability]

In [5]:
llm_with_tools = llm.bind_tools(tools)

In [18]:
result = llm_with_tools.invoke("How will the weather be in munich today?")

from pprint import pprint
pprint(result)
print(type(result))

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tTNjcaLuM6mWDKuG5VlGJx7S', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 84, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CKEeUAy96kklJ1kd3fpcvItoIDq2d', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--09aa0551-a17a-470c-80ca-8acae3a996d6-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Munich'}, 'id': 'call_tTNjcaLuM6mWDKuG5VlGJx7S', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 16, 'total_tokens': 100, 'input_token_details': {'audio': 0

In [7]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': 'call_jznXgqE2FUaL8LP6ox1lsjBi',
  'type': 'tool_call'}]

In [8]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_moX6kOlXF2NVdCeuT7iNgJEQ', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_yl2YdX8KhcIzu8t3xo8KyjwO', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 92, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CKEU18WVZAME8XVI5URsFIPrhhV40', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8d7b5388-db4e-4037-9bdd-6f06f87e3aa7-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'muni

In [9]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': 'call_moX6kOlXF2NVdCeuT7iNgJEQ',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'munich', 'seating_type': 'outdoor'},
  'id': 'call_yl2YdX8KhcIzu8t3xo8KyjwO',
  'type': 'tool_call'}]

In [10]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? Do you still have seats outdoor available?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [11]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MD4QZLlq3Nrfe7EKVNE7XB4S', 'function': {'arguments': '{"location": "Munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_htrO3UQiqQX72DAqZV5iBKTb', 'function': {'arguments': '{"location": "Munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 92, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CKEU20iiq9ci1ForjMdLzdJqa0hVa', 'service_tier': 'default', 'finish

In [19]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability,
}

In [20]:
llm_output.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Munich'},
  'id': 'call_MD4QZLlq3Nrfe7EKVNE7XB4S',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'Munich', 'seating_type': 'outdoor'},
  'id': 'call_htrO3UQiqQX72DAqZV5iBKTb',
  'type': 'tool_call'}]

In [14]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [15]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MD4QZLlq3Nrfe7EKVNE7XB4S', 'function': {'arguments': '{"location": "Munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_htrO3UQiqQX72DAqZV5iBKTb', 'function': {'arguments': '{"location": "Munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 92, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CKEU20iiq9ci1ForjMdLzdJqa0hVa', 'service_tier': 'default', 'finish

In [16]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is 15 degrees Celsius and cloudy. And yes, we still have seats available outdoors. Enjoy your time outside!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 179, 'total_tokens': 208, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CKEU3htUJ6wvT2A1O8SyKQEhQQSPA', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--01e1be1b-4fc7-4648-a3ec-7fe700968c77-0', usage_metadata={'input_tokens': 179, 'output_tokens': 29, 'total_tokens': 208, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})